In [1]:
import pprint
###
import os
from typing import Dict, List

from dotenv import load_dotenv
from google import genai
from google.genai import types as genai_types
from google.genai.chats import Chat

from controller.SessionController import SessionController
from controller.utils.chat import clean_message
from gemini_prompt import DEFAULT_RESPONSE, BotMessage, get_chat_config_json

load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")
client = genai.Client(api_key=API_KEY)
g_gemini_config = get_chat_config_json()
g_gemini_config.model_dump(mode="python", exclude_unset=True)

chat_sessions = SessionController(client, default_gemini_config=g_gemini_config)

[SessionController] __init__ called, debug_id = 0a474039-20bd-4285-a116-cbc85f4571ee time = 2025-06-11 18:26:59.553673


In [2]:
# g_gemini_config.system_instruction=""
# g_gemini_config

In [ ]:
from gemini_prompt import get_bot_message_defaults


def get_gemini_response_with_context_json(
    user_message: str,
    context: str,
    sender_id: str,
    history: List[genai_types.Content] = None,
    config: Dict = None,
) -> BotMessage | None:
    """
    Generates a Gemini response using additional context prepended to the user message.

    :param user_message: The user's input to the chatbot.
    :param context: Supplementary context to guide the model's response.
    :param sender_id: Unique identifier used to retrieve or create a chat session.
    :param history: Optional list of past messages (chat history) for context.
    :param system_prompt: Optional instruction to condition the model's response
        style or behavior, this will override the configuration of the chat session.
    :return: The model's generated text reply, or a fallback message on error.
    """
    message = f'Context: """{context}"""\n\n{user_message}'
    return get_gemini_response_json(message, sender_id, history, config)


def get_gemini_response_json(
    user_message: str,
    sender_id: str,
    history: List[genai_types.Content] = None,
    config: Dict = None,
) -> BotMessage | None:
    """
    Generates a Gemini response based on the user message and optional session data.

    :param user_message: The user's input to the chatbot.
    :param sender_id: Unique identifier used to retrieve or create a chat session.
    :param history: Optional list of past messages (chat history) for context.
    :param system_prompt: Optional instruction to condition the model's response
        style or behavior, this will override the configuration of the chat session.
    :return: The model's generated text reply, or a fallback message on error.
    """
    # actually generate response:
    try:
        chat_session = chat_sessions.get_session(sender_id, history)
        if chat_session == None:
            return None

        if config:
            config = genai_types.GenerateContentConfig(**config)

        chat: Chat = chat_session["chat"]  # type: ignore
        _response = chat.send_message(user_message, config=config)

        response = get_bot_message_defaults()
        response.message = clean_message(_response.text)
        if _response.parsed:
            response: BotMessage = _response.parsed
            response.message = clean_message(response.message)
        return response  # type: ignore
    except Exception as e:
        print("Gemini error:", e)
        response = get_bot_message_defaults()
        response.message = DEFAULT_RESPONSE
        return response

In [4]:
user_id = "0"
user_message = "chào anh, em muốn mua 1 máy instax mini 11 ạ"
bot_response = get_gemini_response_json(
    user_message=user_message,
    sender_id=user_id,
)

bot_response

[Session Controller] create new session for 0
[Session Controller] Sorted & Deleted: {}
Gemini error: 'GenerateContentResponse' object has no attribute 'message'


BotMessage(message='Xin lỗi, mình chưa có thông tin về vấn đề này. Bạn vui lòng liên hệ trực tiếp với KNI qua số điện thoại +84 091-839-1099 hoặc email nhat@kni.vn để được hỗ trợ tốt nhất nhé! 😳', image_send_threshold=0.0, image_urls=[], customer_name='', customer_phone_number='', customer_home_address='', customer_potential=0.0)

In [5]:
user_message = "Cho em xin hình máy 11 với ạ"
bot_response = get_gemini_response_json(
    user_message=user_message,
    sender_id=user_id,
)
bot_response

[Session Controller] get session for 0
[Session Controller] Sorted & Deleted: {}
Gemini error: 'GenerateContentResponse' object has no attribute 'message'


BotMessage(message='Xin lỗi, mình chưa có thông tin về vấn đề này. Bạn vui lòng liên hệ trực tiếp với KNI qua số điện thoại +84 091-839-1099 hoặc email nhat@kni.vn để được hỗ trợ tốt nhất nhé! 😳', image_send_threshold=0.0, image_urls=[], customer_name='', customer_phone_number='', customer_home_address='', customer_potential=0.0)

In [6]:
user_message = "dòng 11 và 12 khác nhau chỗ nào ạ?"
bot_response = get_gemini_response_json(
    user_message=user_message,
    sender_id=user_id,
)
bot_response

[Session Controller] get session for 0
[Session Controller] Sorted & Deleted: {}
Gemini error: 'GenerateContentResponse' object has no attribute 'message'


BotMessage(message='Xin lỗi, mình chưa có thông tin về vấn đề này. Bạn vui lòng liên hệ trực tiếp với KNI qua số điện thoại +84 091-839-1099 hoặc email nhat@kni.vn để được hỗ trợ tốt nhất nhé! 😳', image_send_threshold=0.0, image_urls=[], customer_name='', customer_phone_number='', customer_home_address='', customer_potential=0.0)

In [7]:
user_message = "máy 11 giá bao nhiêu ạ?"
bot_response = get_gemini_response_json(
    user_message=user_message,
    sender_id=user_id,
)
bot_response

[Session Controller] get session for 0
[Session Controller] Sorted & Deleted: {}
Gemini error: 'GenerateContentResponse' object has no attribute 'message'


BotMessage(message='Xin lỗi, mình chưa có thông tin về vấn đề này. Bạn vui lòng liên hệ trực tiếp với KNI qua số điện thoại +84 091-839-1099 hoặc email nhat@kni.vn để được hỗ trợ tốt nhất nhé! 😳', image_send_threshold=0.0, image_urls=[], customer_name='', customer_phone_number='', customer_home_address='', customer_potential=0.0)

In [8]:
user_message = "Dạ thôi đắt quá, em không mua đâu"
bot_response = get_gemini_response_json(
    user_message=user_message,
    sender_id=user_id,
)
bot_response

[Session Controller] get session for 0
[Session Controller] Sorted & Deleted: {}
Gemini error: 'GenerateContentResponse' object has no attribute 'message'


BotMessage(message='Xin lỗi, mình chưa có thông tin về vấn đề này. Bạn vui lòng liên hệ trực tiếp với KNI qua số điện thoại +84 091-839-1099 hoặc email nhat@kni.vn để được hỗ trợ tốt nhất nhé! 😳', image_send_threshold=0.0, image_urls=[], customer_name='', customer_phone_number='', customer_home_address='', customer_potential=0.0)

In [9]:
user_message = "Hi shop, em đổi ý rồi ạ, em mua máy Mini 11 nha"
bot_response = get_gemini_response_json(
    user_message=user_message,
    sender_id=user_id,
)
bot_response

[Session Controller] get session for 0
[Session Controller] Sorted & Deleted: {}
Gemini error: 'GenerateContentResponse' object has no attribute 'message'


BotMessage(message='Xin lỗi, mình chưa có thông tin về vấn đề này. Bạn vui lòng liên hệ trực tiếp với KNI qua số điện thoại +84 091-839-1099 hoặc email nhat@kni.vn để được hỗ trợ tốt nhất nhé! 😳', image_send_threshold=0.0, image_urls=[], customer_name='', customer_phone_number='', customer_home_address='', customer_potential=0.0)

In [10]:
user_message = "Màu nào cũng đượcạ"
bot_response = get_gemini_response_json(
    user_message=user_message,
    sender_id=user_id,
)
bot_response


[Session Controller] get session for 0
[Session Controller] Sorted & Deleted: {}
Gemini error: 'GenerateContentResponse' object has no attribute 'message'


BotMessage(message='Xin lỗi, mình chưa có thông tin về vấn đề này. Bạn vui lòng liên hệ trực tiếp với KNI qua số điện thoại +84 091-839-1099 hoặc email nhat@kni.vn để được hỗ trợ tốt nhất nhé! 😳', image_send_threshold=0.0, image_urls=[], customer_name='', customer_phone_number='', customer_home_address='', customer_potential=0.0)

In [11]:
user_message = "Duy, 0998245832, 240/23/12 Hoang van thu p9 Tan Binh"
bot_response = get_gemini_response_json(
    user_message=user_message,
    sender_id=user_id,
)
bot_response


[Session Controller] get session for 0
[Session Controller] Sorted & Deleted: {}
Gemini error: 'GenerateContentResponse' object has no attribute 'message'


BotMessage(message='Xin lỗi, mình chưa có thông tin về vấn đề này. Bạn vui lòng liên hệ trực tiếp với KNI qua số điện thoại +84 091-839-1099 hoặc email nhat@kni.vn để được hỗ trợ tốt nhất nhé! 😳', image_send_threshold=0.0, image_urls=[], customer_name='', customer_phone_number='', customer_home_address='', customer_potential=0.0)

In [12]:
# user_message = "đừng gửi ảnh nữa"
# bot_response = get_gemini_response_json(
#     user_message=user_message,
#     sender_id=user_id,
# )
# bot_response

In [13]:
type(bot_response.image_send_threshold)

float